In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile6 = pd.read_pickle("BBB_Train_Decile6.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile6.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile6, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 6')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 12s 4ms/step - loss: 0.3731 - acc: 0.8581 - val_loss: 0.3793 - val_acc: 0.8549
Epoch 36/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3716 - acc: 0.8607 - val_loss: 0.3784 - val_acc: 0.8549
Epoch 37/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3688 - acc: 0.8614 - val_loss: 0.3775 - val_acc: 0.8562
Epoch 38/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3729 - acc: 0.8597 - val_loss: 0.3771 - val_acc: 0.8562
Epoch 39/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3725 - acc: 0.8575 - val_loss: 0.3774 - val_acc: 0.8484
Epoch 40/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3742 - acc: 0.8617 - val_loss: 0.3753 - val_acc: 0.8536
Epoch 41/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3698 - acc: 0.8617 - val_loss: 0.3746 - val_acc: 0.8549
Epoch 42/75
3087/3087 [==============================] - 12s

Epoch 20/75
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3411 - acc: 0.8678 - val_loss: 0.3443 - val_acc: 0.8601
Epoch 21/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3362 - acc: 0.8691 - val_loss: 0.3452 - val_acc: 0.8614
Epoch 22/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3384 - acc: 0.8717 - val_loss: 0.3592 - val_acc: 0.8549
Epoch 23/75
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3374 - acc: 0.8659 - val_loss: 0.3413 - val_acc: 0.8614
Epoch 24/75
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3347 - acc: 0.8691 - val_loss: 0.3458 - val_acc: 0.8549
Epoch 25/75
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3393 - acc: 0.8617 - val_loss: 0.3434 - val_acc: 0.8601
Epoch 26/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3404 - acc: 0.8636 - val_loss: 0.3471 - val_acc: 0.8536
Epoch 27/75
3087/3087 [=========================

Epoch 3/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4702 - acc: 0.8105 - val_loss: 0.4449 - val_acc: 0.8109
Epoch 4/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4418 - acc: 0.8218 - val_loss: 0.4273 - val_acc: 0.8212
Epoch 5/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4146 - acc: 0.8390 - val_loss: 0.4177 - val_acc: 0.8187
Epoch 6/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4055 - acc: 0.8442 - val_loss: 0.4045 - val_acc: 0.8251
Epoch 7/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3954 - acc: 0.8523 - val_loss: 0.3945 - val_acc: 0.8420
Epoch 8/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3904 - acc: 0.8497 - val_loss: 0.3869 - val_acc: 0.8407
Epoch 9/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3805 - acc: 0.8575 - val_loss: 0.3864 - val_acc: 0.8381
Epoch 10/100
3087/3087 [========================

Epoch 62/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3378 - acc: 0.8659 - val_loss: 0.3446 - val_acc: 0.8549
Epoch 63/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3381 - acc: 0.8636 - val_loss: 0.3443 - val_acc: 0.8549
Epoch 64/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3283 - acc: 0.8707 - val_loss: 0.3445 - val_acc: 0.8562
Epoch 65/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3320 - acc: 0.8711 - val_loss: 0.3445 - val_acc: 0.8549
Epoch 66/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3270 - acc: 0.8711 - val_loss: 0.3445 - val_acc: 0.8549
Epoch 67/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3332 - acc: 0.8656 - val_loss: 0.3442 - val_acc: 0.8549
Epoch 68/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3351 - acc: 0.8672 - val_loss: 0.3443 - val_acc: 0.8549
Epoch 69/100
3087/3087 [=================

...

Epoch 23/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.4123 - acc: 0.8477 - val_loss: 0.3990 - val_acc: 0.8394
Epoch 24/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4144 - acc: 0.8516 - val_loss: 0.3968 - val_acc: 0.8446
Epoch 25/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4060 - acc: 0.8523 - val_loss: 0.3930 - val_acc: 0.8420
Epoch 26/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3998 - acc: 0.8516 - val_loss: 0.3914 - val_acc: 0.8407
Epoch 27/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4017 - acc: 0.8539 - val_loss: 0.3901 - val_acc: 0.8459
Epoch 28/125
3087/3087 [==============================] - 9s 3ms/step - loss: 0.3931 - acc: 0.8591 - val_loss: 0.3883 - val_acc: 0.8459
Epoch 29/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3944 - acc: 0.8607 - val_loss: 0.3862 - val_acc: 0.8459
Epoch 30/125
3087/3087 [==================

3087/3087 [==============================] - 12s 4ms/step - loss: 0.3692 - acc: 0.8646 - val_loss: 0.3687 - val_acc: 0.8536
Epoch 83/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3664 - acc: 0.8643 - val_loss: 0.3667 - val_acc: 0.8497
Epoch 84/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3693 - acc: 0.8610 - val_loss: 0.3673 - val_acc: 0.8497
Epoch 85/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3692 - acc: 0.8630 - val_loss: 0.3670 - val_acc: 0.8497
Epoch 86/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3618 - acc: 0.8669 - val_loss: 0.3668 - val_acc: 0.8497
Epoch 87/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3750 - acc: 0.8633 - val_loss: 0.3667 - val_acc: 0.8497
Epoch 88/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3673 - acc: 0.8630 - val_loss: 0.3667 - val_acc: 0.8497
Epoch 89/125
3087/3087 [==============================

3087/3087 [==============================] - 8s 3ms/step - loss: 0.3498 - acc: 0.8607 - val_loss: 0.3543 - val_acc: 0.8549
Epoch 17/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3485 - acc: 0.8626 - val_loss: 0.3512 - val_acc: 0.8536
Epoch 18/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3508 - acc: 0.8568 - val_loss: 0.3507 - val_acc: 0.8575
Epoch 19/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3468 - acc: 0.8639 - val_loss: 0.3533 - val_acc: 0.8562
Epoch 20/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3504 - acc: 0.8620 - val_loss: 0.3463 - val_acc: 0.8614
Epoch 21/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3382 - acc: 0.8665 - val_loss: 0.3453 - val_acc: 0.8614
Epoch 22/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3403 - acc: 0.8669 - val_loss: 0.3402 - val_acc: 0.8549
Epoch 23/100
3087/3087 [==============================] - 8s 

3087/3087 [==============================] - 5s 2ms/step - loss: 0.3346 - acc: 0.8614 - val_loss: 0.3371 - val_acc: 0.8601
Epoch 75/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3436 - acc: 0.8604 - val_loss: 0.3368 - val_acc: 0.8601
Epoch 76/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3233 - acc: 0.8682 - val_loss: 0.3372 - val_acc: 0.8601
Epoch 77/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3319 - acc: 0.8711 - val_loss: 0.3375 - val_acc: 0.8588
Epoch 78/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3305 - acc: 0.8672 - val_loss: 0.3372 - val_acc: 0.8601
Epoch 79/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3341 - acc: 0.8730 - val_loss: 0.3368 - val_acc: 0.8601
Epoch 80/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3300 - acc: 0.8707 - val_loss: 0.3368 - val_acc: 0.8601
Epoch 81/100
3087/3087 [==============================] - 5s 

3087/3087 [==============================] - 5s 2ms/step - loss: 0.3408 - acc: 0.8643 - val_loss: 0.3436 - val_acc: 0.8614
Epoch 35/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3477 - acc: 0.8639 - val_loss: 0.3398 - val_acc: 0.8614
Epoch 36/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3414 - acc: 0.8656 - val_loss: 0.3399 - val_acc: 0.8588
Epoch 37/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3535 - acc: 0.8604 - val_loss: 0.3514 - val_acc: 0.8601
Epoch 38/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3411 - acc: 0.8675 - val_loss: 0.3398 - val_acc: 0.8614

Epoch 00038: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 39/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3462 - acc: 0.8630 - val_loss: 0.3388 - val_acc: 0.8614
Epoch 40/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3305 - acc: 0.8688 - val_loss: 0.3385 - v

In [ ]:
0.8297